# Shopping Cart Purchase Item Prediction

## Problem Description
A problem focuses on fashion recommendations. When given user sessions, purchase data and content data about items, can you accurately predict which fashion item will be bought at the end of the session. To better predict, this problem will provide content such as anonymous descriptive labels of the items (such as color, length, neckline, sleeve style, etc.).

## Solution Overview
Our solution uses a graph neural network (GNN) to learn from each session to reveal the underlying connection between viewed items and the final purchased item, the model is developed based on an open-source model and improved by Intel by adding novel ideas including aggregation the item and side information, fusing different predictions, and enhancing accuracy with a weighted loss function. The enhanced model is called 'SIHG4SR'.
![SIHG4SR Architecture](doc/f3-sihg4sr-architecture.png)

## Getting Started

## Step 1: Prepare the dataset
- Download the RecSys Challenge 2022 dataset from the official [website](https://dressipi.com/downloads/recsys-datasets/)
- Put the dataset to your `data_path`

### Step 2: Prepare the environment
```bash
export WORKDIR=`pwd`
export USECASE_PATH=${WORKDIR}"/usecases/2_shopping_cart_purchase_item_pred/"
mkdir -p ${USECASE_PATH}/dataset; cd dataset; unzip dressipi_recsys2022_datasets.zip
```

### Step 3: Training the model

In [1]:
! export USECASE_PATH=`pwd`; export WORKDIR=${USECASE_PATH}/../../; cd ${WORKDIR}/recsys_kit/models/sihg4sr/docker; docker-compose up sihg4sr-train

Starting dgl-cpu-container ... 
Attaching to dgl-cpu-container2mdone
dgl-cpu-container | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
dgl-cpu-container | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
dgl-cpu-container | WARNING: Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv
dgl-cpu-container | prepare kg_feat_dict and kg_feat_cat_dict started ...
dgl-cpu-container | num_unique_features is 904
dgl-cpu-container | prepare kg_feat_dict and kg_feat_cat_dict took 2.6618069447577 s

dgl-cpu-container | Load Train data started ...
dgl-cpu-container | Load Train data took 6.485424380749464 sec
dgl-cpu-container | Load Valid data started ...
dgl-cpu-container | Load Valid data took 0.3543974570930004 sec
dgl-cpu-container | start training
loss is 8.0217:   9%|▉         | 57/627 [02:56<29:18,  3.08s/it]


loss is 7.3406:  19%|█▊        | 117/627 [05:57<25:16,  2.97s/it]


loss is 6.9834:  28%|██▊       | 177/627 [09:00<22:13,  2.96s/it]


loss is 6.7889:  38%|███▊      | 237/627 [12:01<20:13,  3.11s/it]


...

loss is 5.9980:  86%|████████▌ | 537/627 [27:16<04:37,  3.09s/it]


loss is 6.0587:  95%|█████████▌| 597/627 [30:18<01:27,  2.92s/it]


loss is 6.2494: 100%|██████████| 627/627 [31:43<00:00,  3.04s/it]
dgl-cpu-container | 
mrr is 0.181:  32%|███▏      | 31/98 [00:37<01:14,  1.11s/it]


mrr is 0.181:  97%|█████████▋| 95/98 [01:50<00:03,  1.13s/it]


mrr is 0.181: 100%|██████████| 98/98 [01:52<00:00,  1.15s/it]
dgl-cpu-container | Epoch 0: MRR = 18.108%, Hit = 54.602%
100%|██████████| 1/1 [33:37<00:00, 2017.36s/it]
dgl-cpu-container | training completed
dgl-cpu-container | MRR@100	HR@100
dgl-cpu-container | 18.108%	54.602%
dgl-cpu-container exited with code 0


### Step 4: Update new trained Model to Serve directory

```
cp output/saved_models/*.pth model_store/purchase_item_pred/1/
# please clear legacy not wanted models in model_store/purchase_item_pred/1/
```

### Step 5: start the Torchserve inference server
```bash
cd ${WORKDIR}/recsys_kit/serve/triton_serve/docker
docker-compose up triton-server-cpu
```

### Step 6: Client Inference

#### 6.1 Single Request

In [9]:
# 1. prepare request
import pandas as pd
USECASE_PATH="."
valid_dataset = f"{USECASE_PATH}/dataset/test.csv"
valid_df = pd.read_csv(valid_dataset, index_col=0).reset_index(drop=True)

valid_df = valid_df[:1]

print(valid_df.values.tolist())

body = {
  "id" : "1",
  "inputs" : [
      {"name": "input__0", "data": valid_df['session_id'].values.tolist(), "datatype": "INT64", 'shape': [valid_df.shape[0], 1]},
      {"name": "input__1", "data": valid_df['item_id'].values.tolist(), "datatype": "INT64", 'shape': [valid_df.shape[0], 1]},
      {"name": "input__2", "data": valid_df['date'].values.tolist(), "datatype": "BYTES", 'shape': [valid_df.shape[0], 1]},
],
  "outputs" : [{"name" : "session_id"}, {"name": "recommended_items"}]
}

# interact with inference server
import json
import requests
triton_infer_uri = "http://127.0.0.1:8000/v2/models/purchase_item_pred/versions/1/infer"

resp = requests.post(
    triton_infer_uri,
    data=json.dumps(body),
    headers={"content-type": "application/json"},
    proxies={"http": None, "https": None}
)

response_dict = resp.json()
if 'outputs' not in response_dict:
    print(response_dict)
else:
    session_id_out = []
    recommended_items = []
    for item in response_dict['outputs']:
        if item['name'] == 'session_id':
            session_id_out = item['data']
        elif item['name'] == 'recommended_items':
            recommended_items = item['data']
    for session, recommend in zip(session_id_out, recommended_items):
        print(f"For User Session {session}, recommended items are:")
        print(recommended_items)

[[26, 19185, '2021-06-16 09:53:54.158']]
For User Session 26, recommended items are:
[3260, 5383, 12939, 1107, 27921, 6603, 26538, 25188, 20541, 27416, 7368, 11343, 2213, 320, 6021, 9945, 27630, 18093, 21624, 26298, 22896, 20397, 25758, 20889, 16194, 11524, 14040, 15911, 20108, 23239, 15139, 20883, 1483, 21035, 6680, 10610, 19857, 25489, 21484, 4359, 6144, 7719, 26160, 16668, 4551, 11679, 16600, 246, 17514, 25692, 4266, 4112, 16313, 9634, 5874, 3425, 9018, 3858, 15680, 452, 27767, 22203, 21472, 21190, 26124, 21900, 21278, 17599, 5538, 4998, 1589, 10236, 14477, 21824, 21578, 24311, 19480, 2672, 12821, 3880, 823, 11124, 2998, 17586, 7771, 7399, 3102, 6774, 24801, 23497, 27173, 11138, 27388, 2373, 1828, 21856, 15988, 1944, 21928, 16367]


## Advanced - Reproduce Accuracy of ACM Recsys Challenge 2022

### 0. Install with Conda
```
conda create -n sihg4sr python=3.11
conda activate sihg4sr
cd recsys-rekfits; pip install .
cd recsys-refkits/recsys-kits/models/sihg4sr; pip install .
```

### 1. Using Clicks to pretrain the model
```
#augment data for pretrain
python -u src/dataprep/data_prepare.py --data_dir dataset/dressipi_recsys2022_dataset --feat_dir feat/ --save_dir output --train
python -u src/dataprep/data_prepare.py --data_dir output/processed --augment
python -u src/train/cli.py --dataset-dir output/processed/ --feat-dir feat/ --save-path output/saved_models --epochs 10 --use-recent-n-month 5
```

### 2. finetuning upon click pretrained model
```
python -u src/train/cli.py --dataset-dir output/processed/ --feat-dir feat/ --save-path output/saved_models --epochs 4 --finetune --sort-train-data --model {saved_model_name}
```


### 3. using last month for second finetune
```
python -u src/train/cli.py --dataset-dir output/processed/ --feat-dir feat/ --save-path output/saved_models --epochs 2 --finetune --use-recent-n-month 1 --model {saved_model_name}
```
